# Ejercicio de obtener las recetas de ensaladas de recetasgratis.net

In [26]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

### Analizar que pasa cuando hago request a una url que si existe

In [27]:
url = "https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_1.html"
response = requests.get(url)

In [28]:
response.status_code

200

### Analizar que pasa cuando hago request a una url que no existe

In [29]:
url = "https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_1000.html"
response = requests.get(url)

In [30]:
response.status_code

200

In [31]:
response.url

'https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_1.html'

El status code es correcto pero me ha redireccionado a otra url diferente

In [32]:
soup = BeautifulSoup(response.text, "html.parser")

### Obtener las recetas de ensalada de la primera pagina 

In [33]:
recetas_de_una_pagina = soup.find_all("div", attrs={"data-js-selector":"resultado"})
print(len(recetas_de_una_pagina))

50


In [34]:
for receta in recetas_de_una_pagina[0:2]:
    print(receta.get_text().strip()[:100])

Dificultad baja


Receta de Ensalada coleslaw

Dificultad baja


1
15m
Acompañamiento


La ensalada 
Dificultad baja


Receta de Ensalada de quinoa y verduras

Dificultad baja


3
45m
Plato principal




### Crear una función para automatizar

In [67]:
def obtener_recetas_de_una_pagina(url):
    response = requests.get(url)
    if response.status_code == 200 and response.url == url:
        soup = BeautifulSoup(response.text, "html.parser")
        recetas = soup.find_all("div", attrs={"data-js-selector":"resultado"})
        return recetas
    else:
        return -1

In [36]:
url = "https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_1.html"
recetas = obtener_recetas_de_una_pagina(url)
print(len(recetas))

50


In [68]:
url = "https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_2.html"
recetas = obtener_recetas_de_una_pagina(url)
print(len(recetas))

50


### Obtener la info (link, titulo, porciones, tiempo) de una sola receta

In [37]:
url = "https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_1.html"
recetas = obtener_recetas_de_una_pagina(url)
link = np.nan
titulo = np.nan
porciones = np.nan
tiempo = np.nan

try:
    link = receta.find("a",attrs={"class":"titulo"}).attrs['href'].strip()
except:
    pass
try:
    titulo = receta.find("a",attrs={"class":"titulo"}).get_text().strip()
except:
    pass
try:
    porciones = receta.find("span",attrs={"class":"comensales"}).get_text().strip()
except:
    pass
try:
    tiempo = receta.find("span",attrs={"class":"duracion"}).get_text().strip()
except:
    pass

In [38]:
print(titulo, link, porciones, tiempo)

Receta de Ensalada de quinoa y verduras https://www.recetasgratis.net/receta-de-ensalada-de-quinoa-y-verduras-77158.html 3 45m


### Crear una función para automatizar

In [39]:
def obtener_datos_receta(receta):
    
    link = np.nan
    titulo = np.nan
    porciones = np.nan
    tiempo = np.nan
    
    try:
        link = receta.find("a",attrs={"class":"titulo"}).attrs['href'].strip()
    except:
        pass
    try:
        titulo = receta.find("a",attrs={"class":"titulo"}).text.strip()
    except:
        pass
    try:
        porciones = receta.find("span",attrs={"class":"comensales"}).text.strip()
    except:
        pass
    try:
        tiempo = receta.find("span",attrs={"class":"duracion"}).text.strip()
    except:
        pass
        
    return (link, titulo, porciones, tiempo)

In [40]:
obtener_datos_receta(recetas[0])

('https://www.recetasgratis.net/receta-de-ensalada-coleslaw-77205.html',
 'Receta de Ensalada coleslaw',
 '1',
 '15m')

In [41]:
obtener_datos_receta(recetas[4])

('https://www.recetasgratis.net/receta-de-ensalada-de-pasta-con-atun-76932.html',
 'Receta de Ensalada de pasta con atún',
 '5',
 '30m')

### Recorrer las distintas paginas utilizando las funciones anteriores y obtener todas las recetas en un dataframe

In [74]:
lista_de_links = []
lista_de_titulos = []
lista_de_porciones = []
lista_de_tiempos = []

In [77]:
for i in range(1,100):
    url = f"https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_{i}.html"
    recetas = obtener_recetas_de_una_pagina(url)
    if recetas == -1:
        break
    for receta in recetas:
        link, titulo, porciones, tiempo = obtener_datos_receta(receta)
        lista_de_links.append(link)
        lista_de_titulos.append(titulo)
        lista_de_porciones.append(porciones)
        lista_de_tiempos.append(tiempo)

In [76]:
df = pd.DataFrame({"Link":lista_de_links,
                   "Titulo":lista_de_titulos,
                   "Porciones":lista_de_porciones,
                   "Tiempo":lista_de_tiempos})
df

,Link,Titulo,Porciones,Tiempo
0,https://www.recetasgratis.net/receta-de-ensala...,Receta de Ensalada coleslaw,1,15m
1,https://www.recetasgratis.net/receta-de-ensala...,Receta de Ensalada de quinoa y verduras,3,45m
2,https://www.recetasgratis.net/articulo-ensalad...,Ensaladas especiales para fiestas,NaN,NaN
3,https://www.recetasgratis.net/receta-de-ensala...,Receta de Ensalada navideña peruana,10,30m
4,https://www.recetasgratis.net/receta-de-ensala...,Receta de Ensalada de pasta con atún,5,30m
...,...,...,...,...
733,https://www.recetasgratis.net/receta-de-salpic...,Receta de Salpicón de mariscos con Thermomix,6,10m
734,https://www.recetasgratis.net/receta-de-ensala...,Receta de Ensalada de granos,4,30m
735,https://www.recetasgratis.net/receta-de-ensala...,Receta de Ensalada de coditos con queso,2,30m
736,https://www.recetasgratis.net/receta-de-ensala...,Receta de Ensalada de zanahorias con piña y pasas,NaN,NaN
